---
```
Class: DPCV - SDH@HK242 - HCMUT
Name: Nguyễn Lê Nhật Minh
Exercise: 02

```
---

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
# Tải dataset từ file .npy.
# https://drive.google.com/file/d/1ZgUIHN-tM2-W3zfKZnRA0jsq_EDyfxKm/view?usp=sharing
fer13_path = './datasets/fer13_v2.npy'
dataset = np.load(fer13_path, allow_pickle=True).item()

# Chia thành tập train, validation, test.
x_train, y_train = dataset['train']
x_val, y_val = dataset['val']
x_test, y_test = dataset['test']

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

# Chuyển đổi thành tf.data.Dataset để tăng tốc training.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))


def preprocess(image, label):
    # Tiền xử lý ảnh:
    if len(image.shape) == 2:  # (48, 48) → (48, 48, 1)
        image = tf.expand_dims(image, axis=-1)

    image = tf.image.grayscale_to_rgb(image)  # (48, 48, 1) → (48, 48, 3)

    # Resize về kích thước 224x224 (phù hợp với mô hình ImageNet).
    resize_layer = tf.keras.layers.Resizing(224, 224)
    image = resize_layer(image)
    # OR image = tf.image.resize(image, (224, 224))

    # Chuẩn hóa ảnh (Rescale 1./255).
    rescale_layer = tf.keras.layers.Rescaling(1./255)
    image = rescale_layer(image)
    # OR image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Áp dụng data augmentation nếu cần (RandomFlip, RandomRotation, RandomZoom).


# Áp dụng vào dataset
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

# chatGPT gợi ý nên dùng 3 món này
# Without shuffling, the model may overfit.
# Without batching, training might be slow.
# Without prefetching, GPU usage may be inefficient.
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
# Chọn mô hình pretrained

# Dùng tf.keras.applications để load model pretrained trên ImageNet:
# tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
# tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet')
# tf.keras.applications.EfficientNetV2B0(include_top=False, weights='imagenet')
# Bỏ fully connected layers cuối (include_top=False).
# Thêm các layers mới (Flatten, Dense, Dropout) để phù hợp với số lớp của dataset.

# Tham khảo:
# Keras Transfer Learning Guide - https://keras.io/guides/transfer_learning/

In [ ]:
# Freeze / Unfreeze Layers

# Giai đoạn 1: Transfer Learning (Freeze toàn bộ layers)

# Đóng băng (trainable=False) toàn bộ layers của mô hình pretrained.
# Chỉ huấn luyện các layers mới vừa thêm vào.
# Giai đoạn 2: Fine-tuning (Unfreeze một phần layers)

# Mở khóa (trainable=True) một số layer sâu trong mô hình (ví dụ: block cuối của ResNet, MobileNetV2).
# Giảm learning rate (lr=1e-4) để tránh phá vỡ trọng số đã học từ ImageNet.
# Tiếp tục huấn luyện với số epoch nhỏ hơn.

# Tham khảo:
# Fine-tuning Keras models - https://www.tensorflow.org/tutorials/images/transfer_learning#fine-tuning

In [ ]:
# Cấu hình training

# Loss function: tf.keras.losses.SparseCategoricalCrossentropy()
# Optimizer: Adam(learning_rate=0.001)
# Metrics: Accuracy
# Sử dụng EarlyStopping để tránh overfitting.
# Lưu mô hình với ModelCheckpoint.

# Tham khảo:
# Model training with callbacks

In [ ]:
# Đánh giá & Nhận xét

# So sánh kết quả của từng phương pháp:
# Freeze toàn bộ layers (Transfer Learning)
# Unfreeze 1-2 layers cuối (Fine-tuning một phần)
# Unfreeze toàn bộ layers (Fine-tuning toàn bộ mô hình)
# Vẽ biểu đồ loss/accuracy qua từng epoch.
# Đưa ra nhận xét về mô hình tốt nhất dựa trên độ chính xác (accuracy) và khả năng tổng quát hóa (generalization).

# Tham khảo:
# Keras Model Evaluation
